# Initialize Environment

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import numpy as np
import pandas as pd
# ensure kaggle api credentials available via .env
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import shutil
import plotnine as p9
import scipy.special as ssp
import pathlib
import sys

from information_theory.empirical_pmf import empirical_pmf

In [ ]:
DIR_STUDY = pathlib.Path("./information_theory")
DIR_PLOTS = DIR_STUDY / "figures"
DIR_PLOTS.mkdir(exist_ok=True)

In [ ]:
DATA_NEEDS_DOWNLOADED = False

# Extract Data

In [ ]:
if DATA_NEEDS_DOWNLOADED:

    # expected workflow, though authentication issues persist
    # api = KaggleApi()
    # api.authenticate()
    # api.competition_download_file(
    #   "dont-get-kicked", "training.csv", path="./data/dont_get_kicked"
    # )

    os.system('kaggle competitions download -c DontGetKicked')
    shutil.unpack_archive("DontGetKicked.zip", "./data")
    os.remove("DontGetKicked.zip")

In [ ]:
df_train = pd.read_csv("./data/training.csv")
df_test = pd.read_csv("./data/test.csv")

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train.head().transpose()

# Analyze: Discrete-Valued

## Visualize (Intuition)

In [ ]:
df_train['Make'] = df_train["Make"].fillna('NULL')
df_test['Make'] = df_test["Make"].fillna('NULL')

In [ ]:
x_q = df_train['Make']
n_q = df_train.shape[0]

x_p = df_test['Make']
n_p = df_test.shape[0]

In [ ]:
q = empirical_pmf(x_q.copy(), 10, include_other=True)
q['series'] = 'Historical'

In [ ]:
p = empirical_pmf(
    x_p.copy(), 10, categories=list(q['Make']), include_other=True
    )
p['series'] = 'Test'

In [ ]:
compare_p_q = pd.concat([p, q], axis=0)
compare_p_q = compare_p_q.rename(columns={'Make': 'bin'})

In [ ]:
ggp = (
    p9.ggplot(compare_p_q) + 
    p9.theme_minimal() + 
    p9.geom_col(p9.aes('bin', 'prb', fill='series'), position='dodge') + 
    p9.labs(y = "Probability", x = "Make", fill='',
            title=(
                "Visually Insignificant Drift in Make Distribution, \n" +
                "Test vs Historical"
            ),
            caption = (
                "Carvana data, from Kaggle. \n" + 
                "# observations: historical, ~73,000; test, ~49,000."
            )) + 
    p9.theme(legend_position='top', 
             legend_box_spacing=-.1,
             plot_title=p9.element_text(margin={'b': 25})
             ) + 
    p9.coord_flip()
)

In [ ]:
ggp

In [ ]:
ggp.save(DIR_PLOTS / "drift_viz_overall.png", units='in', dpi=300)

## Statistical Tests

In [ ]:
# prefer to align by index, in separate namespaces, for less data manipulation
p = p.set_index('Make')
q = q.set_index("Make")
p = p.loc[q.index]

In [ ]:
# confirm, selection-by-index does not drop any pmf elements
p['prb'].sum()

In [ ]:
kl_div_point = ssp.rel_entr(p["prb"], q["prb"]).sum()
kl_div_point

In [ ]:
def bootstrap_draw_kl_divergence(x_q, nobs_test_set, nobs_lt_pool_other):
    """
    Under the (null) condition that 
    a new observations test set generates from 
    the baseline population probability distribution q:
    bootstrap sample one test set draw, then compute KL Divergence.

    q is unknown and estimable: has generated observations `x_q`.
    """

    var = x_q.name

    q = empirical_pmf(
        x_q, nobs_lt_pool_other, categories=[], include_other=True
        )

    x_p_sample = pd.Series(
        np.random.choice(x_q, size=nobs_test_set, replace=True), name=var
    )
    p = empirical_pmf(
        x_p_sample, nobs_lt_pool_other, 
        categories=list(q[var]), include_other=True
        )
    
    p = p.set_index(var)
    q = q.set_index(var)
    p = p.loc[q.index]

    kl_div = ssp.rel_entr(p["prb"], q["prb"]).sum()

    return {'p': p, 'q': q, 'kl_divergence': kl_div}

In [ ]:
def bootstrap_sampling_distr_kl_divergence(
        x_q, nobs_test_set, nobs_lt_pool_other, n_draws
        ):
    """
    Under the (null) condition that 
    a new observations test set generates from 
    the baseline population probability distribution q:
    simulate sampling distribution of KL Divergence value.

    q is unknown and estimable: has generated observations `x_q`.

    When a new observations test set does truly generate from
    population probability distribution q, 
    KL Divergence sampling variation partly controlled by:
        - Test set sample size (small sample size, wider variation)

    """

    kl_div_draws = [
        bootstrap_draw_kl_divergence(x_q, nobs_test_set, nobs_lt_pool_other)
        for i in range(n_draws)
    ]

    kl_div_values = [x['kl_divergence'] for x in kl_div_draws]
    idx_sort = np.argsort(kl_div_values)
    kl_div_draws = [kl_div_draws[i] for i in idx_sort]

    return kl_div_draws


In [ ]:
N_SAMPLING_DISTR_DRAWS = 1000

### Naive

In [ ]:
kl_div_distr0 = bootstrap_sampling_distr_kl_divergence(
    x_q.copy(), nobs_test_set=df_test.shape[0], 
    nobs_lt_pool_other=10, n_draws=N_SAMPLING_DISTR_DRAWS
    )
kl_div_distr = [x['kl_divergence'] for x in kl_div_distr0]

In [ ]:
kl_div_distr0[0]

In [ ]:
sum(np.array(kl_div_distr) > kl_div_point)

In [ ]:
np.quantile(kl_div_distr, q = [0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
df_p = kl_div_distr0[999]['p'].reset_index(drop=False)
df_p['series'] = 'Extreme Test Draw (49K Obs) from Historical'

df_q = kl_div_distr0[999]['q'].reset_index(drop=False)
df_q['series'] = 'Historical'

compare_p_q = pd.concat([df_p, df_q], axis=0)

In [ ]:
ggp = (
    p9.ggplot(compare_p_q) + 
    p9.theme_minimal() + 
    p9.geom_col(p9.aes('Make', 'prb', fill='series'), position='dodge') + 
    p9.labs(
        y = "Probability", x = "Make", fill='',
        title = (
            "If 49K New Test Observations Exactly Follow Historical, \n" +
            "Will Rarely Observe Drift More Extreme than Below"
        )
    ) + 
    # ensure sufficient white space between title, legend, plot area
    p9.theme(legend_position='top', 
             legend_box_spacing=-.1,
             plot_title=p9.element_text(margin={'b': 25})
             ) + 
    p9.coord_flip()
)

In [ ]:
ggp

In [ ]:
ggp.save(DIR_PLOTS / "drift_viz_boot_naive.png", units='in', dpi=300)

### Calibrated

In [ ]:
kl_div_distr0 = bootstrap_sampling_distr_kl_divergence(
    x_q.copy(), nobs_test_set=100, 
    nobs_lt_pool_other=1, n_draws=N_SAMPLING_DISTR_DRAWS
    )
kl_div_distr = [x['kl_divergence'] for x in kl_div_distr0]

sum(np.array(kl_div_distr) > kl_div_point)

In [ ]:
df_p = kl_div_distr0[950]['p'].reset_index(drop=False)
df_p['series'] = 'Extreme Test Draw (100 Obs) from Historical'

df_q = kl_div_distr0[950]['q'].reset_index(drop=False)
df_q['series'] = 'Historical'

compare_p_q = pd.concat([df_p, df_q], axis=0)

In [ ]:
ggp = (
    p9.ggplot(compare_p_q) + 
    p9.theme_minimal() + 
    p9.geom_col(p9.aes('Make', 'prb', fill='series'), position='dodge') + 
    p9.labs(
        y = "Probability", x = "Make", fill='',
        title = (
            "If 100 New Test Observations Exactly Follow Historical, \n" +
            "Will Rarely Observe Drift More Extreme than Below"
        )
    ) + 
    # ensure sufficient white space between title, legend, plot area
    p9.theme(legend_position='top', 
             legend_box_spacing=-.1,
             plot_title=p9.element_text(margin={'b': 25})
             ) + 
    p9.coord_flip()
)

In [ ]:
ggp

In [ ]:
ggp.save(DIR_PLOTS / "drift_viz_boot_calibrated.png", units='in', dpi=300)

In [ ]:
sys.exit("End of main content.")

# Supplemental

# Analyze: Continuous-Valued

## Visualize (Intuition)

In [ ]:
print(df_train['VehOdo'].isnull().sum())
print(df_test['VehOdo'].isnull().sum())

In [ ]:
x_q = df_train['VehOdo']
n_q = df_train.shape[0]

x_p = df_test['VehOdo']
n_p = df_test.shape[0]

In [ ]:
N_BINS = 10

In [ ]:
q = np.histogram(x_q, bins=N_BINS)
q

In [ ]:
def format_histogram_bin_labels(hist_output):
    """
    Intake np.histogram output directly, or subset bin edges.
    Formatting tailored for magnitudes in 10,000s, 100,000s.
    """
    
    if type(hist_output) == tuple:
        bin_edges = hist_output[1]
    else:
        bin_edges = hist_output

    labels = [
        str(i) + ": " + "[" + f"{bin_edges[i]:,.0f}" + ", " + 
        f"{bin_edges[i+1]:,.0f}" + ")" 
        for i in range(bin_edges.shape[0]-1)
        ]
    # last bin closed on both sides
    labels[-1] = labels[-1].replace(")", "]")
    
    return labels


In [ ]:
bin_labels = format_histogram_bin_labels(q)

In [ ]:
bin_labels

In [ ]:
df_q = pd.DataFrame({
    'n': q[0],
    'prb': q[0] / n_q,
    'series': 'Historical'
})
df_q['bin'] = bin_labels

In [ ]:
p = np.histogram(x_p, bins=q[1])

In [ ]:
p

In [ ]:
df_p = pd.DataFrame({
    'n': p[0],
    'prb': p[0] / n_p,
    'series': 'Test'
})
df_p['bin'] = bin_labels

In [ ]:
compare_p_q = pd.concat([df_q, df_p], axis=0)

In [ ]:
(
    p9.ggplot(compare_p_q) + 
    p9.theme_minimal() + 
    p9.geom_col(p9.aes('bin', 'prb', fill='series'), position='dodge') + 
    p9.labs(y = "Probability", x = "Vehicle Mileage (Binned)", fill='',
            title=(
                "Vehicle Mileage Distribution Doesn't Meaningfully Drift, \n" +
                "Test vs Historical"
            ),
            caption = (
                "Carvana data, from Kaggle. \n" + 
                "# observations: \nhistorical, ~73,000; \ntest, ~49,000."
            )) + 
    p9.theme(
        axis_text_x=p9.element_text(angle=45), 
        plot_caption=p9.element_text(margin={'r': -100, 't': 25}),
        )
)

## Statistical Tests

In [ ]:
kl_div_point = ssp.rel_entr(p[0] / n_p, q[0] / n_q).sum()
kl_div_point

In [ ]:
def bootstrap_draw_kl_divergence(x_q, n_bins, nobs_test_set):
    """
    Under the (null) condition that 
    a new observations test set generates from 
    the baseline population probability distribution q:
    bootstrap sample one test set draw, then compute KL Divergence.

    q is unknown and estimable:
        - Has generated observations `x_q`
        - Estimated by discrete pmf with `n_bins`
    """

    q_hist = np.histogram(x_q, bins=n_bins)
    n_q = q_hist[0].sum()

    x_p_sample = np.random.choice(x_q, size=nobs_test_set, replace=True)
    p_hist = np.histogram(x_p_sample, bins=q_hist[1])

    q_hat = q_hist[0] / n_q
    p_hat = p_hist[0] / nobs_test_set

    kl_div = ssp.rel_entr(p_hat, q_hat).sum()

    out = {
        'p': p_hat, 'n_p': nobs_test_set, 
        'q': q_hat, 'bins': q_hist[1], 
        'kl_divergence': kl_div
        }

    return out

In [ ]:
def bootstrap_sampling_distr_kl_divergence(x_q, n_bins, nobs_test_set, n_draws):
    """
    Under the (null) condition that 
    a new observations test set generates from 
    the baseline population probability distribution q:
    simulate sampling distribution of KL Divergence value.

    q is unknown and estimable:
        - Has generated observations `x_q`
        - Estimated by discrete pmf with `n_bins`

    When a new observations test set does truly generate from
    population probability distribution q, 
    KL Divergence sampling variation partly controlled by:
        - Test set sample size (small sample size, wider variation)
        - Probability distribution q estimate precision 
        (more discretized bins, wider variation) 

    """

    kl_div_draws = [
        bootstrap_draw_kl_divergence(x_q, n_bins, nobs_test_set)
        for i in range(n_draws)
    ]

    kl_div_values = [x['kl_divergence'] for x in kl_div_draws]
    idx_sort = np.argsort(kl_div_values)
    kl_div_draws = [kl_div_draws[i] for i in idx_sort]

    return kl_div_draws


In [ ]:
N_SAMPLING_DISTR_DRAWS = 1000

### Naive

In [ ]:
kl_div_distr0 = bootstrap_sampling_distr_kl_divergence(
    x_q, N_BINS, df_test.shape[0], N_SAMPLING_DISTR_DRAWS
    )
kl_div_distr = [x['kl_divergence'] for x in kl_div_distr0]

In [ ]:
kl_div_distr0[0]

In [ ]:
sum(np.array(kl_div_distr) > kl_div_point)

In [ ]:
np.quantile(kl_div_distr, q = [0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
# delta = [kl_div_distr[i] - kl_div_distr[i-1] for i in range(1, len(kl_div_distr))]
# sum(np.array(delta) < 0)

In [ ]:
df_p = pd.DataFrame({
    'prb': kl_div_distr0[999]['p'], 'series': 'Extreme Test Draw (49K Obs) from Historical'
    })
df_p['bin'] = format_histogram_bin_labels(kl_div_distr0[0]['bins'])

df_q = pd.DataFrame({'prb': kl_div_distr0[999]['q'], 'series': 'Historical'})
df_q['bin'] = format_histogram_bin_labels(kl_div_distr0[1]['bins'])

compare_p_q = pd.concat([df_p, df_q], axis=0)

In [ ]:
(
    p9.ggplot(compare_p_q) + 
    p9.theme_minimal() + 
    p9.geom_col(p9.aes('bin', 'prb', fill='series'), position='dodge') + 
    p9.labs(
        y = "Probability", x = "Vehicle Mileage (Binned)", fill='',
        title = (
            "If 49K New Test Observations Follow Historical Distribution, \n" +
            "Will Rarely Observe Drift More Extreme than Below"
        )
    ) + 
    # ensure sufficient white space between title, legend, plot area
    p9.theme(legend_position='top', 
             legend_box_spacing=-.1,
             plot_title=p9.element_text(margin={'b': 25}),
             axis_text_x=p9.element_text(angle=45))
)

### Calibrated

In [ ]:
kl_div_distr0 = bootstrap_sampling_distr_kl_divergence(
    x_q, N_BINS, 100, N_SAMPLING_DISTR_DRAWS
    )
kl_div_distr = [x['kl_divergence'] for x in kl_div_distr0]

sum(np.array(kl_div_distr) > kl_div_point)

In [ ]:
df_p = pd.DataFrame({
    'prb': kl_div_distr0[999]['p'], 'series': 'Extreme Test Draw (100 Obs) from Historical'
    })
df_p['bin'] = format_histogram_bin_labels(kl_div_distr0[0]['bins'])

df_q = pd.DataFrame({'prb': kl_div_distr0[999]['q'], 'series': 'Historical'})
df_q['bin'] = format_histogram_bin_labels(kl_div_distr0[1]['bins'])

compare_p_q = pd.concat([df_p, df_q], axis=0)

In [ ]:
(
    p9.ggplot(compare_p_q) + 
    p9.theme_minimal() + 
    p9.geom_col(p9.aes('bin', 'prb', fill='series'), position='dodge') + 
    p9.labs(
        y = "Probability", x = "Vehicle Mileage (Binned)", fill='',
        title = (
            "If 100 New Test Observations Follow Historical Distribution, \n" +
            "Will Rarely Observe Drift More Extreme than Below"
        )
    ) + 
    # ensure sufficient white space between title, legend, plot area
    p9.theme(legend_position='top', 
             legend_box_spacing=-.1,
             plot_title=p9.element_text(margin={'b': 25}),
             axis_text_x=p9.element_text(angle=45))
)